# Regional sea ice concentration and thickness for CMIP6 large ensembles

### Author - Chris Wyburn-Powell, see the latest version on [github](https://github.com/chrisrwp/low-frequency-variability/blob/main/input_data/Regional_sea_ice_CMIP6.ipynb)


**Input:**
- Sea ice concentration (SIC) from variable `siconc` or `siconca` and sea ice thickness (SIT) from variable `sithick` (note `sivol` is unavailible for some models).  Historical forcings are used for 1850-2014 for the following CMIP6 models which provide large ensembles with member numbers in parentheses:
    - CanESM5 (65 for siconc, 40 for sithick)
    - MIROC6 (50)
    - GISS-E2-1-G (43)
    - IPSL-CM6A-LR (32)
    - CNRM-CM6-1 (30)
    - NorCPM1 (21 - 9 more awaited)
- Regridded MASIE regions for each model

**Output:**
- SIA for each region, 1850-2014
- Average SIC for each region, 1850-2014
- SIV for each region, 1850-2014
- Average SIT for each region, 1850-2014
- Pan-Arctic SIA, 1850-2014
- Pan-Arctic SIV, 1850-2014
- Linear detrended and 2 year lowpass filter of regional SIC and SIT, 1920-2014

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
import datetime
import os
import re
import dask
print(datetime.datetime.now())

2022-07-22 14:02:12.298156


In [131]:
#the following dask workers are sufficient for processing around 100 members
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(cores    = 1,
                     memory   = '8GB',
                     queue    = 'casper',
                     walltime = '00:30:00',
                     project='UCUB0084')

cluster.scale(5)
client = Client(cluster)
client

## Make useful lists

In [117]:
#list of model names with CVDP >=30 members
model_names  = ['CanESM5', 'MIROC6', 'GISS-E2-1-G', 'IPSL-CM6A-LR',
                'CNRM-CM6-1', 'NorCPM1'
]

model_centers = {
    'CanESM5':'CCCma', 'MIROC6':'MIROC', 'GISS-E2-1-G':'NASA-GISS',
    'IPSL-CM6A-LR':'IPSL', 'CNRM-CM6-1':'CNRM-CERFACS', 'NorCPM1':'NCC',
}

areacello_paths = {
    'CanESM5': '/glade/collections/cmip/CMIP6/ScenarioMIP/CCCma/CanESM5/'\
        +'ssp585/r1i1p1f1/Ofx/areacello/gn/v20190429/areacello/'\
        +'areacello_Ofx_CanESM5_ssp585_r1i1p1f1_gn.nc',
    
    'MIROC6': '/glade/collections/cmip/CMIP6/CMIP/MIROC/MIROC6/historical/'\
        +'r1i1p1f1/Ofx/areacello/gn/v20190311/areacello/'\
        +'areacello_Ofx_MIROC6_historical_r1i1p1f1_gn.nc',
    
    'GISS-E2-1-G': '/glade/collections/cmip/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/'\
        +'piControl/r1i1p1f1/Ofx/areacello/gn/v20180824/areacello/'\
        +'areacello_Ofx_GISS-E2-1-G_piControl_r1i1p1f1_gn.nc',
    
    'IPSL-CM6A-LR': '/glade/collections/cmip/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/'\
        +'historical/r1i1p1f1/Ofx/areacello/gn/v20180803/areacello/'\
        +'areacello_Ofx_IPSL-CM6A-LR_historical_r1i1p1f1_gn.nc',
    
    'CNRM-CM6-1': '/glade/collections/cmip/CMIP6/CMIP/CNRM-CERFACS/CNRM-CM6-1/'\
        +'historical/r1i1p1f2/Ofx/areacello/gn/v20180917/areacello/'\
        +'areacello_Ofx_CNRM-CM6-1_historical_r1i1p1f2_gn.nc', 
    
    'NorCPM1': '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
        +'masie_masks/areacello_Ofx_NorCPM1_piControl_r1i1p1f1_gn.nc',
    
}

lat_names = {'CanESM5': 'latitude', 'MIROC6':'latitude', 'GISS-E2-1-G':'lat',
             'IPSL-CM6A-LR': 'nav_lat', 'CNRM-CM6-1':'lat', 
             'NorCPM1': 'latitude',
}

x_y_names = {'CanESM5':['i','j'], 'MIROC6':['x','y'], 
             'GISS-E2-1-G':['lat','lon'], 'IPSL-CM6A-LR':['x','y'], 
             'CNRM-CM6-1':['x','y'], 'NorCPM1':['i','j'],
}

doi_dict = {'CanESM5':'10.5194/gmd-12-4823-2019', 
            'MIROC6':'10.5194/gmd-12-2727-2019',
            'GISS-E2-1-G':'10.1029/2019MS002025',
            'IPSL-CM6A-LR':'10.1029/2019MS002010',
            'CNRM-CM6-1':'10.1029/2019MS001683',
            'NorCPM1':'10.5194/gmd-12-343-2019',
}

In [4]:
#make a list of variant IDs availible for each model in CVDP
CVDP_cmip6_hist_mem_siconc = {}
CVDP_cmip6_hist_list = np.sort(np.array(os.listdir(
    '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
    +'CMIP6_CVDP_historical')))

for model_name in model_names:
    temp_mem_list = []
    
    for i in CVDP_cmip6_hist_list:
        #search for a substring beginning with the model name and '_' then
        #return substring between that and 'cvdp'
        if re.search('(?<={}_)(.*)(?=cvdp)'.format(model_name), i):
            #if such a string is found, append it to the list
            temp_mem_list.append(re.search('(?<={}_)(.*)(?=cvdp)'.format(
                model_name), i)[0][:-1])
    
    CVDP_cmip6_hist_mem_siconc[model_name] = np.ravel(temp_mem_list)

In [5]:
#replace variant IDs for CNRM-CM6-1, removing those yet unavailible for siconc
#or for sithick. Note CanESM5 has missing sithick for the 25 r<x>i1p2f1
#variant labels, but these will just not be computed regionally
CVDP_cmip6_hist_mem_siconc['CNRM-CM6-1'] = np.ravel(
    ['r10i1p1f2', 'r11i1p1f2', 'r12i1p1f2', 'r13i1p1f2', 'r14i1p1f2',
    'r15i1p1f2', 'r16i1p1f2', 'r17i1p1f2', 'r18i1p1f2', 'r19i1p1f2',
    'r1i1p1f2', 'r20i1p1f2', 'r29i1p1f2', 'r2i1p1f2', 'r3i1p1f2', 
    'r4i1p1f2', 'r5i1p1f2', 'r6i1p1f2', 'r7i1p1f2', 'r8i1p1f2', 'r9i1p1f2']
)

## Define function for loading data

In [30]:
def load_member_CMIP6(model, ripf, output_var, chunk=False):
    '''
    Open a single member file of either sea ice concentration or thickness
    from glade using xarray.open_dataset
    
    Parameters
    ----------
    model: string
        The name of the model e.g. CanESM5
    ripf : string,
        r<>: realisation (i.e. ensemble member), i<>: initialisation method, 
        p<>: physics, f<>: forcing, e.g. r15i1p2f1
    output_var : string,
        Variable of concentration of volume e.g. 'siconc', 'sivol', 'sithick'
    chunk : bool, optional
        Open with dask chunks (auto chunked) if true, do not use dask is false
    
    Returns
    ----------
        xarray.DataSet object from glade sea ice output
    '''  
    
    multi_file = False #assume a single file unless otherwise for certain models
    
    base_path = '/glade/collections/cmip/CMIP6/CMIP/'\
        +'{}/{}/historical/{}/'.format(model_centers[model], model, ripf)\
        +'SImon/{}/'.format(output_var)
    
    ############### generate the file path ###############
    
    ##### CanESM5 #####
    if model == 'CanESM5':
        path = base_path+'gn/v20190429/{}/{}'.format(output_var, output_var)\
            +'_SImon_CanESM5_historical_{}_gn_185001-201412.nc'.format(ripf)
    
    ##### MIROC6 #####
    elif model == 'MIROC6':
        if ripf[2] == 'i' or ripf[1:3] == '10': 
            i = '20181212'
        else:
            i = '20200519'
                
        multi_file = True
        
        path = [
            base_path+'gn/v{}/{}/{}'.format(i, output_var, output_var)\
            +'_SImon_MIROC6_historical_{}_gn_185001-194912.nc'.format(ripf),
            base_path+'gn/v{}/{}/{}'.format(i, output_var, output_var)\
            +'_SImon_MIROC6_historical_{}_gn_195001-201412.nc'.format(ripf),
        ]
        
    ##### GISS-E2-1-G #####
    elif model == 'GISS-E2-1-G':
        if ripf[-4:] == 'p3f1': 
            i = '20190702'
        elif ripf[1:4] in ['101', '102']:
            i = '20190815'
        elif ripf[-4:] == 'p1f1' and ripf[0:3] in ['r1i', 'r2i']:
            i = '20180827'
        elif ripf[-4:] == 'p1f1' and ripf[1] in ['3', '4', '5']:
            i = '20180828'
        elif ripf[-4:] == 'p1f1' and ripf[1] in ['6', '7']:
            i = '20180829'      
        elif ripf[-4:] == 'p1f1' and ripf[0:3] in ['r8i', 'r9i', 'r10']:
            i = '20180830'      
        elif ripf[-4:] in ['p1f2', 'p1f3']:
            i = '20190903'  
        elif ripf[-4:] == 'p5f1':
            i = '20190905'    
        else: 
            print('invalid variant ID')
        
        multi_file = True

        path = [
            base_path+'gn/v{}/{}/{}_'.format(i, output_var, output_var)\
            +'SImon_GISS-E2-1-G_historical_{}_gn_'.format(ripf)\
            +'185001-190012.nc',
            base_path+'gn/v{}/{}/{}_'.format(i, output_var, output_var)\
            +'SImon_GISS-E2-1-G_historical_{}_gn_'.format(ripf)\
            +'190101-195012.nc',
            base_path+'gn/v{}/{}/{}_'.format(i, output_var, output_var)\
            +'SImon_GISS-E2-1-G_historical_{}_gn_'.format(ripf)\
            +'195101-200012.nc',
            base_path+'gn/v{}/{}/{}_'.format(i, output_var, output_var)\
            +'SImon_GISS-E2-1-G_historical_{}_gn_'.format(ripf)\
            +'200101-201412.nc',
        ]

    ##### IPSL-CM6A-LR #####
    elif model == 'IPSL-CM6A-LR':
        if ripf[:4] == 'r32i':
            i = '90802'
        else:
            i = '80803'

        path = base_path+'gn/v201{}/{}/{}_'.format(i, output_var, output_var)\
            +'SImon_IPSL-CM6A-LR_historical_{}_gn_185001-201412.nc'.format(ripf)
    
    ##### CNRM-CM6-1 #####
    elif model == 'CNRM-CM6-1':
        if ripf == 'r1i1p1f2':
            i = '20180917'
        elif ripf == 'r2i1p1f2':
            i = '20181126'
        elif int(ripf[1]) in np.arange(3,10) or ripf[1:3] == '10':
            i = '20190125'
        elif int(ripf[1:3]) in np.arange(11,21):
            i = '20191004'
        elif ripf in ['r29i1p1f1', 'r29i1p1f2']:
            i = '20200529'
        else:
            print('invalid variant ID')

        path = base_path+'gn/v{}/{}/{}_'.format(i, output_var, output_var)\
            +'SImon_CNRM-CM6-1_historical_{}_gn_185001-201412.nc'.format(ripf)
    
    ##### NorCPM1 #####
    elif model == 'NorCPM1':
        if ripf[1:3] in ['15', '16', '22', '23', '26', '28', '30'] and \
            output_var == 'sithick':
            i = '20190914'
        else:
            i = '20200724'
            
        path = base_path+'gn/v{}/{}/{}_'.format(i, output_var, output_var)\
            +'SImon_NorCPM1_historical_{}_gn_185001-201412.nc'.format(ripf)
        
    else:
        print('invalid model name')
       
    ########## use the file path to open the NetCDF file using xarray ##########
    if chunk:
        if multi_file:
            data = xr.open_mfdataset(path, combine='nested', concat_dim='time',
                                     chunks='auto')
        else: #note CanESM5 doesn't like auto selection of chunk sizes
            data = xr.open_dataset(path, chunks={'time':198})
    else:
        if multi_file:
            data = xr.open_mfdataset(path, combine='nested', concat_dim='time')
        else:
            data = xr.open_dataset(path)
            
    return(data)

## Make the region masks and define location of areacello files

In [75]:
#make reduced areacello files for all models
for model_name in model_names:

    areacello = xr.open_dataset(areacello_paths[model_name])
    areacello_30N = areacello['areacello'].where(
        areacello[lat_names[model_name]]>30)

    areacello_30N.attrs = areacello.attrs.copy()
    areacello_30N.to_netcdf(
        '/glade/work/cwpowell/low-frequency-variability/'\
        +'raw_data/masie_masks/areacello_{}_30N.nc'.format(model_name))

**Regrid the MASIE region mask for each of the model grids, using nearest neighbor for the areacello files**:
`cdo remapnn,<areacello_path> /glade/work/cwpowell/low-frequency-variability/raw_data/masie_masks/masiemask_ims4km.nc /glade/work/cwpowell/low-frequency-variability/raw_data/masie_masks/masiemask_<model_name>.nc`

## Calculate the regional sea ice concentration and thickness with `dask`

In [28]:
def regional_calc_dask(model_name, mem, var_names):
    '''
    Calculate regional SIA, SIC, SIV, SIT from glade files of siconc and sithick
    from historical CMIP6 model runs. Regions based on NSIDC MASIE regions.

    Parameters
    ----------
    model_name: string
        The name of the model e.g. CanESM5
    mem: string
        Variant ID e.g. 'r1ip1f1' where the letters correspond to:
        r<>: realisation (i.e. ensemble member),
        i<>: initialisation method, p<>: physics, f<>: forcing
    var_names: list of strings
        List of the concentration and volume or thickness variables names
        e.g. ['siconc','sivol'] or ['siconca','sithick']

Returns
    ----------
        xarray.Dataset with variables of:
        regional SIA, regional average SIC, regional SIV, regional average SIT,
        pan-Arctic SIA, pan-Arctic SIV
    '''  
    
    #load the areacello file and assign nan to latitudes below 30N    
    areacello = areacello_['areacello'].where(
        areacello_[lat_names[model_name]]>30)
    
    ##################### load the data files #####################
    #load the SIC and SIT data and select above 30N
    SIC = load_member_CMIP6(model_name, mem, var_names[0], chunk=True)
    #remove concentration values below 30N and convert to fraction from %
    SIC = SIC[var_names[0]].where(
        areacello[lat_names[model_name]]>30, drop=False)/100
  
    #load the sithick if availible
    if var_names[1] == 'sithick':
        no_sithick = False
        SIT = load_member_CMIP6(model_name, mem, 'sithick', chunk=True)
        SIT = SIT['sithick'].where(
            areacello[lat_names[model_name]]>30, drop=False)
    else:
        no_sithick = True

    #replace time with datetime64 for consistency across models
    SIC['time'] = xr_new_time.sel(time=slice('1850', '2014'))
    
    if no_sithick==False:
        SIT['time'] = xr_new_time.sel(time=slice('1850', '2014'))

    ############## compute SIA, SIV and average SIC and SIT ################
    #only do the calculation where there is sea ice
    SIC = SIC.where(SIC>0)
    
    if no_sithick==False:
        SIT = SIT.where(SIT>0)
    
    #calculate the pan-Arctic SIA and SIV
    pan_Arctic_SIA = (SIC * areacello).sum(x_y_names[model_name][0]).sum(
        x_y_names[model_name][1])
    if no_sithick==False:
        pan_Arctic_SIV = (SIC * SIT * areacello).sum(
            x_y_names[model_name][0]).sum(x_y_names[model_name][1])
    else:
        pan_Arctic_SIV = pan_Arctic_SIA * np.nan
    
    #calculate regional data
    SIA_regions = []
    SIC_regions_av = []
    SIV_regions = []
    SIT_regions_av = []

    for region_ in np.arange(1,17):
        SIC_region = SIC.where((region_mask['regions']==region_))
        SIA_region = (SIC_region*areacello).sum(x_y_names[model_name][0]).sum(
            x_y_names[model_name][1])

        area_region = areacello.where(region_mask['regions']==region_).sum()
        
        SIA_regions.append(SIA_region)
        SIC_regions_av.append(SIA_region / area_region)
        
        if no_sithick==False:
            SIT_region = SIT.where((region_mask['regions']==region_))

            SIV_region = (SIT_region * SIC * areacello).sum(
                x_y_names[model_name][0]).sum(x_y_names[model_name][1])

            SIV_regions.append(SIV_region)
            SIT_regions_av.append(SIV_region / area_region)
        
        else: #append nan values of the same shape as the SIC derived data
            SIV_regions.append(SIA_region*np.nan)
            SIT_regions_av.append(SIA_region*np.nan)
            
    
    #concatenate all regions
    SIA_regions =  xr.concat((SIA_regions),dim='region')
    SIA_regions['region'] = np.arange(1,17)
    SIC_regions_av = xr.concat((SIC_regions_av),dim='region')
    SIC_regions_av['region'] = np.arange(1,17)

    SIV_regions = xr.concat((SIV_regions),dim='region')
    SIV_regions['region'] = np.arange(1,17)
    SIT_regions_av = xr.concat((SIT_regions_av),dim='region')
    SIT_regions_av['region'] = np.arange(1,17)

    
    return(
        xr.Dataset(
            {'regional_SIA':SIA_regions, 'regional_SIC':SIC_regions_av,
             'regional_SIV':SIV_regions, 'regional_SIT':SIT_regions_av,
             'Arctic_SIA':pan_Arctic_SIA, 'Arctic_SIV':pan_Arctic_SIV
            }
        ).drop('type')
    )

In [109]:
#loop through a few members at a time, loading data in dask chunks and then
#computing a concatenated xarray.Dataset with a couple of members' data at once

model_name = 'NorCPM1'
print(model_name)

###load data common for all variant IDs
#load data containing a datetime64 time series
xr_new_time = xr.open_dataset(
    '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
    +'datetime64_1850_2100_monthly.nc')
xr_new_time = xr_new_time['time'].sel(time=slice('1850','2014'))

#load the region masks
region_mask = xr.open_dataset(
    '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
    +'masie_masks/masiemask_{}.nc'.format(model_name)
)

#load the areacello file
areacello_ = xr.open_dataset(
        '/glade/work/cwpowell/low-frequency-variability/'\
        +'raw_data/masie_masks/areacello_{}_30N.nc'.format(model_name)
    )

for mem_start in np.arange(0, len(CVDP_cmip6_hist_mem_siconc[model_name]), 2):
    print(datetime.datetime.now(), mem_start)
    #make an array of xarray.Datasets with dask chunks and wait to calculate
    all_mem_data = []
    for mem_ in CVDP_cmip6_hist_mem_siconc[model_name][mem_start:mem_start+2]:
        
        ##uncomment the 4 lines below to run with CanESM5 as only 40/65 members
        ##have sithick but 65/65 have siconc currently
        # if mem_[-3] == '1':
        #     all_mem_data.append(regional_calc_dask(model_name, mem_, 
        #                                            ['siconc','none']))
        # else:
        all_mem_data.append(regional_calc_dask(model_name, mem_, 
                                               ['siconc','sithick']))

    all_mem_model_concat = xr.concat((all_mem_data),dim='member')
    all_mem_model_concat['member'] = CVDP_cmip6_hist_mem_siconc[model_name][
        mem_start:mem_start+2]

    print(datetime.datetime.now(), 'start compute')
    #only now do the calculations of both members at the same time for all vars
    to_save = all_mem_model_concat.compute()

    print(datetime.datetime.now(), 'end compute')

    to_save.to_netcdf('/glade/work/cwpowell/low-frequency-variability/'\
                      +'raw_data/regional_sea_ice_CMIP6/Regional_SIC_SIT_'\
                      +'{}_mem_{}_{}.nc'.format(
                          model_name, str(mem_start).zfill(2),
                          str(mem_start+1).zfill(2))
                     )

In [113]:
#now combine each of the 1 or 2 member files together for each model
for model_name in model_names:

    all_model_files = []
    for file in os.listdir('/glade/work/cwpowell/low-frequency-variability/'\
                           +'raw_data/regional_sea_ice_CMIP6/'):
        if model_name in file:
            temp_file = xr.open_dataset(
                '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
                +'regional_sea_ice_CMIP6/'+file)
            all_model_files.append(temp_file)

    all_model_xr = xr.concat((all_model_files),dim='member')
    all_model_xr.to_netcdf('/glade/work/cwpowell/low-frequency-variability/'\
                           +'raw_data/regional_sea_ice_CMIP6/Regional_SIC_'\
                           +'SIT_all_mem_{}_1850_2014.nc'.format(model_name))

## Detrend and extract 2 year lowpass filter

In [114]:
def filt_lowpass(data_, sample_freq, cutoff, order, ax_n, detrend=False,
                 standard=False):
    '''
    Filter a time series using a lowpass Butterworth filter. 
    Uses scipy.signal.butter and scipy.signal.filtfilt
    
    Parameters
    ----------
    data_ : n dimensional xarray dataarray,
        Data to detrend and/or standardize, which can contain nans
    sample_freq: float,
        The sampling frequency of the input data, typically sample_freq=1 [year]
    cutoff: float,
        The fraction of the nyquist frequency (itself half the sampling 
        frequency). To filter with a 2-year lowpass filter with
        sample_freq=1 (year), cutoff=0.25
    order: int
        The order of the Butterworth filter, typically 4-6
    ax_n : int
        Which axis to do the filtering on (time)
    detrend: bool
        Whether to detrend the data with a linear trend
    standard: bool
        Whether to standardize the data after filtering

    Returns
    ----------
        numpy array of the same shape as the input data
    '''

    if detrend: #detrend the data first
        data_ = (data_ * 0) + signal.detrend(data=data_.fillna(0), axis=ax_n)

    b, a = signal.butter(order, cutoff, btype='lowpass') #low pass filter
    #apply the filter forward and backward along a given axis
    filtered = signal.filtfilt(b, a, data_, axis=ax_n) 

    filtered_xr = (data_ * 0) + filtered

    if standard: #standardize the data
        filtered_xr = (filtered_xr - filtered_xr.mean('time')) \
                      / filtered_xr.std('time')

    return(filtered_xr)

In [130]:
for model_name in model_names:
    
    unfiltered = xr.open_dataset(
        '/glade/work/cwpowell/low-frequency-variability/raw_data/'\
        +'regional_sea_ice_CMIP6/Regional_SIC_SIT_all_mem_'\
        +'{}_1850_2014.nc'.format(model_name)
    )

    all_month_SIC = []
    all_month_SIT = []

    for month_ in np.arange(1,13):
        #select regional average SIC data for each month, and limit to 1920-2014
        unfiltered_SIC = unfiltered['regional_SIC'].sel(
            time=unfiltered['time.month']==month_).sel(
            time=slice('1920','2014'))*100

        unfiltered_SIT = unfiltered['regional_SIT'].sel(
            time=unfiltered['time.month']==month_).sel(
            time=slice('1920','2014'))

        #filter with a 2 year lowpass filter for each month
        all_month_SIC.append(filt_lowpass(unfiltered_SIC, 1, 0.25, 5, 2, 
                                           detrend=True, standard=False)
                             )
        all_month_SIT.append(filt_lowpass(unfiltered_SIT, 1, 0.25, 5, 2, 
                                           detrend=True, standard=False)
                             )

    model_xr = xr.Dataset(
        {'SIC': xr.concat((all_month_SIC), dim='time').sortby('time'),
         'SIT': xr.concat((all_month_SIT), dim='time').sortby('time')
        }
    )
    
    model_xr.attrs = {
        'Description': '2 year lowpass filter of linearly detrended '\
            +'regional average sea ice concentration (SIC) in % and sea ice '\
            +'thickness (SIT) in meters for the climate model '\
            +'{} for each month for 1920-2014 with '.format(model_name)\
            +'historical forcing. Regions as defined for NSIDC MASIE-NH '\
            +'Version 1, doi:10.7265/N5GT5K3K.',
        'Timestamp'  : str(datetime.datetime.utcnow().strftime(
            "%H:%M UTC %a %Y-%m-%d")),
        'Data source': 'CMIP6 model output for siconc and sithick from'\
             +'{}, doi:{}.'.format(model_name, doi_dict[model_name]),
        'Analysis'   : 'https://github.com/chrisrwp/low-fequency-variability/'\
            +'input_data/Regional_sea_ice_CMIP6.ipynb'
    }

    model_xr.to_netcdf('/glade/work/cwpowell/low-frequency-variability/'\
        +'input_data/Regional_SIC_SIT_detrended_lowpass_'\
        +'{}_1850_2014.nc'.format(model_name)
                      )